# “一站式”因子探索性分析 (Factor EDA) - v2.0

本notebook完全基于`ResearchContext`和`AlphaDataTool`，提供一个与AlphaHome主系统无缝集成的、现代化的因子分析流程。

**核心流程:**
1. **初始化 `ResearchContext`**: 作为所有数据和服务的统一入口。
2. **执行研究流水线**: 复用`main.py`中定义的自动化流程，直接得到计算好的因子和未来收益率。
3. **交互式分析与可视化**: 对流水线产出的结果进行更深入、灵活的探索。

In [ ]:
# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
import sys

warnings.filterwarnings('ignore')

# 设置中文字体和图表风格
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

# 添加项目根目录到Python路径，以便导入alphahome模块
# 假设此notebook在 '.../research/projects/your_project/notebooks/' 下运行
project_root = Path.cwd().parents[3]
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"项目根目录已添加: {project_root}")

## 1. 初始化与数据加载

我们将通过`ResearchContext`来获取所有需要的服务和数据。

In [ ]:
from research.tools.context import ResearchContext
from main import create_factor_research_pipeline

# 初始化 ResearchContext
# research project的根目录是notebooks目录的上一级
project_path = Path.cwd().parent
context = ResearchContext(project_path=project_path)

# 检查配置是否加载成功
print("研究项目配置加载成功:")
display(context.config['project'])

# 使用 data_tool 获取股票列表
stock_list = context.data_tool.get_stock_list()[:20] # 选择20只股票作为示例
print(f"\n成功获取 {len(stock_list)} 只股票用于分析。")

## 2. 执行自动化研究流水线

我们将直接复用 `main.py` 中定义的 `create_factor_research_pipeline` 函数来创建并运行流水线。这个流水线会自动完成数据加载、因子计算、未来收益率计算等所有预处理工作。

In [ ]:
# 创建流水线
pipeline = create_factor_research_pipeline(context)

# 获取流水线初始参数 (来自config.yml)
initial_params = context.config.get('pipeline_params', {})
initial_params['stock_list'] = stock_list # 使用我们刚刚选择的股票列表

# 运行流水线
pipeline.run(initial_params=initial_params)

# 从流水线结果中提取最终的、包含所有因子的DataFrame
final_result = pipeline.results[-1]['output']
df_final = final_result.get('factor_data')

print("自动化流水线执行完成！")
if df_final is not None:
    print(f"最终数据集包含 {len(df_final)} 条记录，{len(df_final.columns)} 个字段。")
    display(df_final.head())
else:
    print("最终数据集为空，请检查流水线各步骤的输出。")

## 3. 交互式因子分析

现在，我们已经有了一个干净、完整的数据集 `df_final`，其中包含了原始价格、计算好的因子和未来收益率。接下来，我们可以像之前一样，进行深入的交互式分析。

In [ ]:
if df_final is not None and not df_final.empty:
    from research.templates.database_research.src.steps import FactorICAnalysisStep, FactorQuantileAnalysisStep

    # 提取因子列
    factor_cols = list(context.config.get('factors', {}).keys())

    print("开始对以下因子进行交互式分析:")
    print(factor_cols)
else:
    print("数据为空，跳过交互式分析。")

### 3.1 因子IC分析 (复现流水线步骤)

我们可以手动调用分析步骤，对单个或多个因子进行分析。

In [ ]:
if df_final is not None and not df_final.empty:
    ic_step = FactorICAnalysisStep(context, factor_cols=factor_cols)
    ic_step.run(factor_data=df_final)

    # 也可以从上下文中获取之前流水线运行的结果
    ic_results = context.get_analysis_result('ic_analysis')
    print("\nIC分析结果摘要:")
    for factor, result in ic_results.items():
        print(f"- 因子: {factor}, IC均值: {result['IC均值']:.4f}, ICIR: {result['ICIR']:.4f}")

### 3.2 因子分位数分析 (复现流水线步骤)

手动调用分位数分析步骤，可以方便地更换要分析的因子。

In [ ]:
if df_final is not None and not df_final.empty:
    # 对 'ma_5' 因子进行5分位分析
    if 'ma_5' in factor_cols:
        quantile_step_ma5 = FactorQuantileAnalysisStep(context, factor_col='ma_5', quantiles=5)
        quantile_step_ma5.run(factor_data=df_final)

    # 对 'rsi_14' 因子进行10分位分析
    if 'rsi_14' in factor_cols:
        quantile_step_rsi14 = FactorQuantileAnalysisStep(context, factor_col='rsi_14', quantiles=10)
        quantile_step_rsi14.run(factor_data=df_final)

### 3.3 因子相关性分析

In [ ]:
if df_final is not None and not df_final.empty:
    factor_corr = df_final[factor_cols].corr()

    plt.figure(figsize=(10, 8))
    sns.heatmap(factor_corr, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title('因子相关性矩阵')
    plt.show()